<a href="https://colab.research.google.com/github/AxelADN/Perceptual_System_V5/blob/main/Perceptual_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!mkdir Test_Data
!mkdir Test_Data/Scenes
!mkdir Test_Data/Scenes_Test
!mkdir Test_Data/PCD_Objects
!mkdir Data_Out
!mkdir Hipp
!mkdir Hipp/VCC
!mkdir Hipp/VCC_Test

In [ ]:
!unzip Test_Data/Cocina_1.zip
!mv Cocina_1 Test_Data/Cocina_1
!rm Test_Data/Cocina_1.zip

In [ ]:
!rm -fr Test_Data/Cocina_1/

======================================================= IMPORTS ==================================================

In [ ]:
import cv2
import pickle
cv2.__version__
import sys
import os
from google.colab.patches import cv2_imshow
import numpy as np
np.set_printoptions(threshold=False)
import csv
import pandas as pd
import statistics as st



---


=========================================== GENERAL CLASSES & FUNCTIONS ==========================================

In [ ]:
def save(obj,fileName):
    with open(fileName, "wb") as f:
        pickle.dump(obj,f,protocol=pickle.HIGHEST_PROTOCOL)
    #del obj

def load(fileName):
    with open(fileName,"rb") as f:
        return pickle.load(f)

In [ ]:
class Point:

    totalPoints =  0

    def __init__(self,x0,y0):
        self.x = x0
        self.y = y0
        Point.totalPoints += 1

    def __str__(self):
        return f"[ {self.x}, {self.y} ]"

    def __eq__(self,other):
        if not isinstance(other, Point):
            return False
        return self.x == other.x and self.y == other.y

    def __hash__(self):
        return hash((self.x, self.y, 31, Point.totalPoints))

In [ ]:
class ROI:
    def __init__(self,maskX0=(-1,-1), maskY0=(-1,-1),origin0=(-1,-1),size0=(-1,-1)):
        self.maskX = maskX0
        self.maskY = maskY0
        self.origin = origin0
        self.size = size0
        if self.maskX == (-1,-1) or self.maskY == (-1,-1):
            self.maskX = (self.origin[0],self.origin[0]+self.size[0]-1)
            self.maskY = (self.origin[1],self.origin[1]+self.size[1]-1)
        if self.origin == (-1,-1) and self.size != (-1,-1):
            self.origin = (self.maskX[0],sskYelf.maskY[0])
            self.size = (self.maskX[1]-self.maskX[0], self.maskY[1]-self.maskY[0])

    def x(self):
        return self.maskX

    def y(self):
        return self.maskY


In [ ]:
class BagOfWords:
    def __init__(self, alphabet0=(None,), featureArray0=[], posID0 = (-1,-1), th0=125):
        self.posID = posID0
        self.th = th0
        self.alphabet = alphabet0
        self.featureArray = featureArray0
        if None in self.alphabet:
            self.alphabet = tuple([f.symbol for f in self.featureArray])
        if self.posID == (-1,-1):
            self.posID = self.featureArray[-1].pos
        self.currentSymbol = self.alphabet[0]
        if len(self.featureArray) <= 0:
            self.codebook = {}
        else:
            self.codebook = self.generateCodebook()
        pass

    def generateCodebook(self):
        this_codebook = {}
        i = 0
        for matrix in self.featureArray:
            this_codebook[self.alphabet[i]] = np.count_nonzero(matrix.matrix)
            i+=1
        return this_codebook

    def diff(self, bow0, ord0 = 1):
        dist = []
        for s in self.codebook.keys():
            #print(s)
            dist.append(self.codebook[s]-bow0.codebook[s])
        return dist
        #this = np.array(this)
        #other = np.array(other)
        #return np.linalg.norm(this-other, ord=ord0)

    def hashID(self):
        listK = list(self.codebook.keys())
        listK.sort()
        return tuple([self.codebook[s] for s in listK])

    def size(self):
        return len(self.alphabet)


    def isEmpty(self):
        return len(self.codebook) == 0 or None in self.codebook.keys() or not any(list(self.codebook.values()))

    def printCodebook(self):
        print(f'Name: {self.featureArray[-1].name}:')
        for s in self.codebook.keys():
            print(f'    pos: {self.posID} --> {self.codebook[s]}')



In [ ]:
class GridMask:
    def __init__(self,sizePerSquare0=(-1,-1),sizeXY0=(-1,-1),sizeOverall0=(-1,-1)):
        self.sizePerSquare = sizePerSquare0
        self.sizeXY = sizeXY0
        self.sizeOverall = sizeOverall0
        if self.sizePerSquare == (-1,-1):
            self.sizePerSquare = (int(self.sizeOverall[0]/self.sizeXY[0]),int(self.sizeOverall[1]/self.sizeXY[1]))
        self.roiArray = self.buildRois()

    def buildRois(self):
        roiArray = []mean
        for i in range(self.sizeXY[0]):
            for j in range(self.sizeXY[1]):
                ox = i*self.sizePerSquare[0]
                oy = j*self.sizePerSquare[1]
                roiArray.append(ROI(origin0=(ox,oy),size0=self.sizePerSquare))
        return roiArray

    def mask(self):
        return self.roiArray


In [ ]:
class AdjMatrix:

    def __init__(self, bows0=None, size0=(-1,-1), numSymbols0=3):
        self.hv = ""
        self.bows = bows0
        self.size = size0
        self.numSymbols = numSymbols0
        self.adj = np.array([])
        self.figureID = np.array([])
        self.direction = np.array([])

    def assignID(self):
        sh = self.adj.shape
        self.figureID = np.reshape(self.adj,(sh[0],sh[1]*sh[2]))
        self.figureID = np.reshape(self.figureID,sh[0]*sh[1]*sh[2])
        print(self.figureID)
        return self.figureID

    def buildAdjMatrix(self):
        self.adj = np.zeros((self.numSymbols,self.size[0]*self.size[0],self.size[1]*self.size[1]),dtype=np.int_)
        print(self.adj.shape)
        for i in range(len(self.bows)):
            for j in range(len(self.bows)):
                if self.bows[i] is not self.bows[j]:
                    if not self.bows[i].isEmpty() and not self.bows[j].isEmpty():
                        if self.checkOrder(i,j):
                            self.adj[:,j,i] = np.array(self.bows[j].diff(self.bows[i]))
                            #print(self.adj[j][i])
        self.minMax()

    def minMax(self):
        self.direction = np.sign(self.adj)
        adjAbs = np.absolute(self.adj)
        max_min_ref = np.max(adjAbs)-np.min(adjAbs)
        if max_min_ref == 0:
            minmax = np.zeros_like(adjAbs,dtype=np.float_)
        else:
            minmax = (adjAbs-np.min(adjAbs))/max_min_ref
        self.adj = minmax#np.multiply(minmax,self.direction)

    def checkOrder(self,i0,j0):
        if self.hv == "H":
            return self.bows[i0].posID[0] < self.bows[j0].posID[0]
        if self.hv == "V":
            return self.bows[i0].posID[1] < self.bows[j0].posID[1]

    def printID(self):
        s = self.hv + ' - START\n'
        #print(len(self.adj))
        i=0
        for row in self.figureID:
            s += '['
            for n in row:
                s += f'{n},'
            s += '],\n'
        s += ']\n' + '===================================================' + '\n'
        print(s)

    def printAdj(self):
        s = 'START'
        #print(len(self.adj))
        i=0
        for ch in self.adj:
            s+=f'{i}: ['
            for row in ch:
                s += '['
                for n in row:
                    s += f'{n},'
                s += '],\n'
            s += ']\n' + '===================================================' + '\n'
            i += 1
        print(s)

class AdjMatrixH(AdjMatrix):

    def __init__(self, bows1=None, size1=(-1,-1), numSymbols1=3):
        super().__init__(bows0=bows1,size0=size1,numSymbols0=numSymbols1)
        self.hv = "H"
        self.buildAdjMatrix()
        self.assignID()


class AdjMatrixV(AdjMatrix):

    def __init__(self, bows1=None, size1=(-1,-1), numSymbols1=3):
        super().__init__(bows0=bows1,size0=size1,numSymbols0=numSymbols1)
        self.hv = "V"
        self.buildAdjMatrix()
        self.assignID()

In [ ]:
class ConcurrenceMap:
    def __init__(self,bowArray0=[]):
        self.bowArray = bowArray0
        self.map = {}
        self.buildMap()

    def buildMap(self):
        for bow_i in self.bowArray:
            if not bow_i.isEmpty():
                id_i = bow_i.hashID()
                self.map[id_i] = [bow_i,]
                for bow_j in self.bowArray:
                    if not bow_j.isEmpty():
                        id_j = bow_j.hashID()
                        if id_i  != id_j:
                            self.map[id_i].append(bow_j)

    def add(self, bow0, bow1):
        id_0 = bow0.hashID()
        if self.map[id_0] != None:
            self.map[id_0] = []
        self.map[id_0].append(bow1)

    def get(self, bow0):
        id_0 = bow0.hashID()
        return self.map[id_0]

    def printMap(self):
        s = ""
        for bow_id_i in self.map.keys():
            s += f'{bow_id_i} --> ['
            for bow_j in self.map[bow_id_i]:
                bow_id_j = bow_j.hashID()
                s += f'{bow_id_j}, '
            s += f'],\n'
        print(s)




---

============================================= VISUAL CORTICES ====================================

In [ ]:
def getFeatures(imgName0):

    img_array = cv2.imread(imgName0, cv2.IMREAD_GRAYSCALE)
    img_array = cv2.resize(img_array,(100,100))

    orb = cv2.ORB_create(nfeatures=1500)

    keypoints, descriptors = orb.detectAndCompute(img_array, None)
    #print(f"Num_KeyPoints -- > {len(keypoints)}")

    #img_points = cv2.drawKeypoints(img_array, keypoints, None)
    #cv2_imshow(img_points)
    #cv2.waitKey(0)
    #cv2.destroyAllWindows()



    return img_array,keypointsskY

In [ ]:
def getImgData(imgName0):
    img_array = cv2.imread(imgName0, cv2.IMREAD_COLOR)
    img_array = cv2.resize(img_array,(100,100))

    return img_array

In [ ]:
def getFeaturesColorBased(img0):
    img0 = cv2.cvtColor(img0, cv2.COLOR_BGR2HSV)
    (H,S,V) = cv2.split(img0)

    H_thresh = cv2.adaptiveThreshold(H,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,11,2)
    S_thresh = cv2.adaptiveThreshold(S,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,11,2)
    V_thresh = cv2.adaptiveThreshold(V,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,11,2)

    img0 = cv2.cvtColor(img0,cv2.COLOR_HSV2BGR)
    (B,G,R) = cv2.split(img0)

    B_thresh = cv2.adaptiveThreshold(B,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,11,2)
    G_thresh = cv2.adaptiveThreshold(G,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,11,2)
    R_thresh = cv2.adaptiveThreshold(R,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,11,2)

    img0 = cv2.cvtColor(img0,cv2.COLOR_BGR2LAB)
    (L,A,BB) = cv2.split(img0)

    L_thresh = cv2.adaptiveThreshold(L,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,11,2)
    A_thresh = cv2.adaptiveThreshold(A,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,11,2)
    BB_thresh = cv2.adaptiveThreshold(BB,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,11,2)

    img0 = cv2.cvtColor(img0,cv2.COLOR_LAB2BGR)
    img0 = cv2.cvtColor(img0,cv2.COLOR_BGR2YCrCb)
    (Y,Cr,Cy) = cv2.split(img0)

    Y_thresh = cv2.adaptiveThreshold(Y,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,11,2)
    Cr_thresh = cv2.adaptiveThreshold(Cr,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,11,2)
    Cy_thresh = cv2.adaptiveThreshold(Cy,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,11,2)

    return {
        'H': H_thresh,
        'S': S_thresh,
        'V': V_thresh,
        'B': B_thresh,
        'G': G_thresh,
        'R': R_thresh,
        'L': L_thresh,
        'A': A_thresh,
        'BB': BB_thresh,
        'Y': Y_thresh,
        'Cr': Cr_thresh,
        'Cy': Cy_thresh
        }

In [ ]:
def getFeaturesRetinaBased(img0,name0="NAME"):

    retina = cv2.bioinspired_Retina.create((img0.shape[1], img0.shape[0]))
    retina.write('Test_Data/retinaParams.xml')
    retina.setup('Test_Data/retinaParams.xml')
    retina.run(img0)
    parvo = retina.getParvo()
    magno = retina.getMagno()
    parvo = cv2.cvtColor(parvo,cv2.COLOR_BGR2LAB)
    (L,A,B) = cv2.split(parvo)
    name1 = name0.split("/")[-1][:-4]
    #print(name1)


    L_thresh = cv2.adaptiveThreshold(L,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,11,2)
    A_thresh = cv2.adaptiveThreshold(A,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,11,2)
    B_thresh = cv2.adaptiveThreshold(B,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,11,2)
    M_thresh = cv2.adaptiveThreshold(magno,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,11,2)

    if not cv2.imwrite("Hipp/VCC_IMG/"+name1+"_L"+".png",L_thresh):
        raise Exception("Could no write Image")
    if not cv2.imwrite("Hipp/VCC_IMG/"+name1+"_A"+".png",A_thresh):
        raise Exception("Could no write Image")
    if not cv2.imwrite("Hipp/VCC_IMG/"+name1+"_B"+".png",B_thresh):
        raise Exception("Could no write Image")
    if not cv2.imwrite("Hipp/VCC_IMG/"+name1+"_M"+".png",M_thresh):
        raise Exception("Could no write Image")


    return {'L': L_thresh, 'A': A_thresh, 'B': B_thresh, 'M': M_thresh}

In [ ]:
class FeatureMatrix:
    def __init__(self,size0=(100,100), rawMatrix0=np.array([0]), symbol0=None, name0=None,pos0=(-1,-1)):
        self.symbol = symbol0
        self.size = size0
        self.matrix = rawMatrix0.astype(np.bool_) #np.zeros(self.size,dtype=np.bool_)
        self.name = name0
        self.pos = pos0
        pass

    def subMatrix(self, roi=ROI()):
        return FeatureMatrix(size0=roi.size,rawMatrix0=self.matrix[roi.y()[0]:roi.y()[1],roi.x()[0]:roi.x()[1]],symbol0=self.symbol,name0=self.name,pos0=roi.origin)

    def __str__(self):
        s = ""
        for pt in self.matrix:
            s += str(pt)
            s += '\n'
        return s

    def printMatrix(self):
        s = ""
        if not np.any(self.matrix):
            s += "[EMPTY]"
        else:
            s += "["
            for row in self.matrix:
                s += "["
                for pt in row:
                    if pt:
                        s += "1"
                    else:
                        s += "0"
                s += "],\n"
            s += "]"
        print(s)

In [ ]:
def getDataNames(directory0, ext0="png"):
    imgs = []
    aux = []
    for root, dirs, files in os.walk("/content/"+directory0, topdown=True):
        for file in files:
            if file.split(".")[-1] == ext0:
                aux.append(os.path.join(root,file))
        imgs += aux
        aux = []
    return imgs

In [ ]:
def getData(data_img0,save0="Hipp/VCC/"):

    #print(data_img)
    data = {}

    for img_name in data_img0:
        #print(img_name)
        bin = []
        img_array = getFeaturesRetinaBased(getImgData(img_name),name0=img_name)
        for ch in img_array.keys():
            bin.append(FeatureMatrix(rawMatrix0=img_array[ch],symbol0=ch, name0=img_name.split("/")[-1][:-4]))
            #bin[-1].printMatrix()
        data[img_name] = bin

    for s in data.keys():
        #print(s)
        save(data[s],os.path.join(save0,s.split("/")[-1][:-4]))



In [ ]:
data_img = getDataNames("Test_Data/Cocina_1")
getData(data_img)

In [ ]:
!rm -rf Hipp/VCC_IMG/*

----------------TEST OBJECTS-----------------

In [ ]:
class PCD():
    def __init__(self,img0,img_name0):
        self.img = img0
        self.shape = self.img.shape
        self.img_name = img_name0
        self.PCD_Bundle = {}
        self.getPCDs()

    def savePCDs(self,dir0):
        for level in self.PCD_Bundle.keys():
            name = "PCD_"+self.img_name.split("/")[-1].split(".")[0]+"_"+str(level)+"."+self.img_name.split("/")[-1].split(".")[1] #os.path.join(dir0,"PCD_"+self.img_name.split("/")[-1]+"_"+str(level))
            path = os.path.join(dir0,name)
            print(path)
            if not cv2.imwrite(path,self.PCD_Bundle[level]):
                raise Exception("Could no write Image")

    def getPCDs(self,levelArray0=[50,25,20,10,5,4,2,1]):
        for level in levelArray0:
            if level > 1:
                self.PCD_Bundle[level] = self.convertToPCD(level)
            else:
                self.PCD_Bundle[level] = self.img.copy()

    def convertToPCD(self,level0):
        pcd_img = np.zeros(self.shape)
        grid = GridMask(sizeXY0=(int(self.img.shape[0]/level0),int(self.img.shape[1]/level0)),sizeOverall0=self.img.shape)
        for mask in grid.mask():
            sub_matrix = self.img[mask.y()[0]:mask.y()[1],mask.x()[0]:mask.x()[1],:]
            av_arr = (np.average(sub_matrix[:,:,0]),np.average(sub_matrix[:,:,1]),np.average(sub_matrix[:,:,2]))
            for ch in range(self.shape[2]):
                for i in range(mask.x()[0],mask.x()[1]+1):
                    for j in range(mask.y()[0],mask.y()[1]+1):
                        pcd_img[j,i,ch] = av_arr[ch]
        return pcd_img


    def _convertToPCD(self,level0):
        size = self.img.shape[0]
        factor = (size-(level0-1))/size #(100-(100-1))/100
        small_size = (int(size*factor),int(size*factor))
        interpolation = cv2.INTER_NEAREST
        small_img = cv2.resize(self.img,small_size,interpolation)
        normal_size = (size,size)
        normal_img = cv2.resize(small_img,normal_size,interpolation)
        return normal_img



In [ ]:
for img_name in data_img:
    pcd = PCD(getImgData(img_name),img_name)
    pcd.savePCDs("/content/Test_Data/PCD_Objects")



In [ ]:
!zip -r Test_Data/PCD_Objects.zip Test_Data/PCD_Objects/

In [ ]:
!rm Test_Data/PCD_Objects/*

In [ ]:
data_img = getDataNames("Test_Data/PCD_Objects")
getData(data_img,save0="Hipp/VCC_Test")



---



======================================================= LOC ========================================================

In [ ]:
def getVccData(dir0):
    filenames = os.listdir(dir0)
    data = {}
    for filename in filenames:
        data[filename] = load(os.path.join(dir0,filename))
    return data

def LOC_buildBOWs(data0,partitionSize0=(20,20)):
    partitionSize = partitionSize0
    multiMatrix = {}
    bags = {}
    mask = GridMask(sizeOverall0=next(iter(data0.values()))[-1].size,sizeXY0=partitionSize)
    for s in data0.keys():
        bags[s] = []
        multiMatrix[s] = []
        for roi in mask.mask():
            #print(f'({roi.y()}, {roi.x()})')
            multiMatrix[s].append([])
            for ch in range(len(data0[s])):
                multiMatrix[s][-1].append(data0[s][ch].subMatrix(roi))
                #multiMatrix[s][-1][-1].printMatrix()
        for subMatrix in multiMatrix[s]:
            bags[s].append(BagOfWords(featureArray0=subMatrix))
            #bags[s][-1].printCodebook()
    return bags

def LOC_buildAdj(bags0,size0=(20,20)):

    adjH = {}
    adjV = {}

    for s in bags0.keys():
        adjH[s] = AdjMatrixH(bows1=bags0[s],size1=size0,numSymbols1=bags0[s][0].size())
        adjV[s] = AdjMatrixV(bows1=bags0[s],size1=size0,numSymbols1=bags0[s][0].size())
        #print(s)
        #adjV[s].printID()
        #adjH[s].printAdj()
        #print(np.count_nonzero(adjH[s].figureID))
    return adjH,adjV



In [ ]:
del VCC_Data
del LOC_bags
del LOC_adjH
del LOC_adjV
VCC_Data = getVccData("Hipp/VCC")
LOC_bags = LOC_buildBOWs(VCC_Data)
LOC_adjH,LOC_adjV = LOC_buildAdj(LOC_bags)

In [ ]:
for s in VCC_Data.keys():
    cv2.imwrite("Hipp/VCC_IMG/"+s+".png",VCC_Data[s][2].matrix*255)

----------------TEST OBJECTS-----------------

In [ ]:
del VCC_Data_Test
del LOC_bags_Test
del LOC_adjH_Test
del LOC_adjV_Test
VCC_Data_Test = getVccData("Hipp/VCC_Test")
LOC_bags_Test = LOC_buildBOWs(VCC_Data_Test)
LOC_adjH_Test,LOC_adjV_Test = LOC_buildAdj(LOC_bags_Test)

----------------TEST MATCH-----------------

In [ ]:
res_arr = {}
for pcd_s in LOC_adjH_Test.keys():
    res_arr[pcd_s] = {}
    pcd_adjH_id = LOC_adjH_Test[pcd_s].assignID()
    pcd_adjV_id = LOC_adjV_Test[pcd_s].assignID()
    #print(pcd_adjH_id)
    for s in LOC_adjH.keys():
        c_adjH_id = LOC_adjH[s].assignID()
        c_adjV_id = LOC_adjV[s].assignID()
        normH = np.linalg.norm(pcd_adjH_id-c_adjH_id)
        normV = np.linalg.norm(pcd_adjV_id-c_adjV_id)
        res_arr[pcd_s][s] = np.linalg.norm((normH+normV)/2)



In [ ]:
for pcd_s in res_arr.keys():
    print(max(list(res_arr[pcd_s].values())))
    #for s in res_arr[pcd_s].keys():


In [ ]:
df_Test = pd.DataFrame(columns=['Object',1,2,4,5,10,20,25,50])

In [ ]:
def getValuesManual(res_arr0,s0,level0,mean={},test_names=[]):
    values = []
    for s_aux in res_arr0["PCD_"+s0+"_"+str(level0)].keys():
        this_val = res_arr0["PCD_"+s0+"_"+str(level0)][s_aux]
        #print("Val"+str(this_val))
        if s_aux in test_names:
            if mean:
                #print(mean)
                this_val -= mean[s_aux][level0]*1
        else:
            this_val += 0
        values.append(this_val)
    return values

def getValuesAuto(res_arr0,s0,level0):
    return list(res_arr0["PCD_"+s0+"_"+str(level0)].values())

for test in names_test.keys():
    levels = [50,25,20,10,5,4,2,1]
    sd = {}
    mean = {}
    for s in res_arr['PCD_D_Cocina_1_Escurridor_1_1'].keys():
        sd[s] = {}
        mean[s] = {}
        for level in levels:
            max_count = 0
            max_values = []
            values = getValuesManual(res_arr,s,level,mean2,names_test[test])
            mean[s][level] = st.mean(values)
            #print(f"MEAN: {median}")
            for val in values:
                if val <= mean[s][level]:
                    max_count += 1
                    max_values.append(val)
            #if len(max_values) == 1:
            #    sd[s][level] = 0
            #elif len(max_values) == 0:
            #    sd[s][level] = -1
            #else:
            sd[s][level] = max_count#/np.std(np.array(max_values)) #max_count
    #mean2 = mean.copy()

    df_Test.loc[len(df_Test.index)] = [test]+[sd[test][v] for v in levels[::-1]]


print(df_Test)



                    Object   1   2   4   5  10  20  25  50
0      D_Cocina_1_Tetera_1   4   5   5   5   8   9   8  10
1    F_Cocina_1_Gabinete_1  11  13  13  12  14  14  13  15
2    F_Cocina_1_Gabinete_9   7   7   7   7   7   8   8  11
3      D_Cocina_1_Sarten_1   8   8   8   8   8   8   7   9
4    F_Cocina_1_Gabinete_2   9   9   9   8   8   9   8  10
5   F_Cocina_1_Extractor_1   8   8   8   7   7   5   8  11
6    D_Cocina_1_Parrilla_1   4   4   8   5   7   9   8  12
7      D_Cocina_1_Sarten_2   6   6   6   6   7   9   6  11
8    F_Cocina_1_Gabinete_3  10  10  10  11  11  10   7  11
9    F_Cocina_1_Gabinete_8   9  12  13  11  10  12  12  13
10    D_Cocina_1_Manzana_1   4   4   3   4   3   4   6  12
11     F_Cocina_1_Estufa_1   3   4   6   9  12  13  12  13
12   F_Cocina_1_Gabinete_7  11  11  11  11  11  11  11  11
13      F_Cocina_1_Refri_1  13  13  13  13  11  11  11  11
14   F_Cocina_1_Gabinete_6  12  12  14  13  15  16  13  15
15   F_Cocina_1_Gabinete_5   7   9   9   9   9   9  10  

In [ ]:
row_mean = df_Test.mean()
print(df_Test.mean())
row_mean['Object'] = 'Mean'
df_Test.loc[len(df_Test)]= row_mean
row_var = df_Test.var()
row_var['Object'] = 'Variance'
df_Test.loc[len(df_Test)]= row_var
df_Test.to_csv('Data_Out/objects_vs_pcdLevel_Test.csv', index=False)

1      7.833333
2      8.388889
4      8.833333
5      8.611111
10     9.166667
20     9.777778
25     9.222222
50    11.555556
dtype: float64


<ipython-input-378-aab90e25fd12>:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  row_mean = df_Test.mean()
<ipython-input-378-aab90e25fd12>:2: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(df_Test.mean())
<ipython-input-378-aab90e25fd12>:5: FutureWarning: The default value of numeric_only in DataFrame.var is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  row_var = d

In [ ]:
this_level = 10
values = [sd[x][this_level] for x in sd.keys()]
print(values)
values.sort(reverse=False)
values = [l for l in values if l > 0]
print(values)
min_vals = [y for y in sd.keys() if sd[y][this_level] in values ]

for s in min_vals:#sd.keys():
    for lvl in levels:
        if lvl == this_level:
            print(f'{s} - LVL:{lvl} --> sd: {sd[s][lvl]}')

In [ ]:
def convertToDataArray(data0):
    data_array = []
    av = {}
    for obj in data0.keys():
        data_dir = {}
        data_dir["Object"] = obj
        last_val = 0
        if type(data0[obj]) is type(dict()):
            for level in levels[::-1]:
                if str(level) not in av.keys():
                    av[str(level)] = []
                av[str(level)] = []
                data_dir[str(level)] = data0[obj][level]
                if data_dir[str(level)] <= 0:
                    data_dir[str(level)] = last_val
                last_val = data_dir[str(level)]
                av[str(level)].append(last_val)

            data_array.append(data_dir)
    for level in av.keys():
        av_arr = np.array(av[level])
        av[level] = np.average(av_arr)
    data_array.append(av)
    return data_array

In [ ]:
def getObjectFromDataArray(data0,name0):
    gotData = []
    for data_i in data0:
        if "Object" in data_i.keys():
            #print(data_i["Object"])
            if data_i["Object"] == name0:
                gotData.append(data_i)

    return gotData



In [ ]:
df = pd.DataFrame(convertToDataArray(sd))[:-1]
df = df.loc[df['Object'].isin(['D_Cocina_1_Tetera_1', 'F_Cocina_1_Gabinete_1', 'F_Cocina_1_Gabinete_9', 'D_Cocina_1_Sarten_1', 'F_Cocina_1_Gabinete_2', 'F_Cocina_1_Extractor_1', 'D_Cocina_1_Parrilla_1', 'D_Cocina_1_Sarten_2', 'F_Cocina_1_Gabinete_3', 'F_Cocina_1_Gabinete_8', 'D_Cocina_1_Manzana_1', 'F_Cocina_1_Estufa_1', 'F_Cocina_1_Gabinete_7', 'F_Cocina_1_Refri_1', 'F_Cocina_1_Gabinete_6', 'F_Cocina_1_Gabinete_5', 'D_Cocina_1_Pan_1', 'F_Cocina_1_Gabinete_4'])]
row_mean = df.mean()
print(df.mean())
row_mean['Object'] = 'Mean'
df.loc[len(df)]= row_mean
row_var = df.var()
row_var['Object'] = 'Variance'
df.loc[len(df)]= row_var
df.to_csv('Data_Out/objects_vs_pcdLevel.csv', index=False)

1     14.000000
2     16.166667
4     16.944444
5     17.000000
10    18.000000
20    19.888889
25    19.555556
50    18.111111
dtype: float64


<ipython-input-340-f9f632e18419>:3: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  row_mean = df.mean()
<ipython-input-340-f9f632e18419>:4: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  print(df.mean())
<ipython-input-340-f9f632e18419>:7: FutureWarning: The default value of numeric_only in DataFrame.var is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  row_var = df.var()


In [ ]:

pd.DataFrame(
    getObjectFromDataArray(
        convertToDataArray(sd)
        ,name0="D_Cocina_1_Licuadora_1"
        )
        ).to_csv('Data_Out/Test_PCD_Object_Blender.csv', index=False)




---



======================================================= TOS ========================================================

In [ ]:
class SceneMap:
    def __init__(self,name0="Test_Data/Scenes"):
        self.nameMap = [
            'F_Cocina_1_Gabinete_1',    #1
            'F_Cocina_1_Gabinete_2',    #2
            'F_Cocina_1_Gabinete_3',    #3
            'F_Cocina_1_Gabinete_4',    #4
            'F_Cocina_1_Gabinete_5',    #5
            'F_Cocina_1_Gabinete_6',    #6
            'F_Cocina_1_Gabinete_7',    #7
            'F_Cocina_1_Gabinete_8',    #8
            'F_Cocina_1_Gabinete_9',    #9
            'F_Cocina_1_Extractor_1',   #10
            'F_Cocina_1_Extractor_2',   #11
            'F_Cocina_1_Extractor_3',   #12
            'F_Cocina_1_Estufa_1',      #13
            'F_Cocina_1_Estufa_2',      #14
            'F_Cocina_1_Estufa_3',      #15
            'F_Cocina_1_Refri_1',       #16
            'F_Cocina_1_Refri_2',       #17
            'F_Cocina_1_Refri_3',       #18
            'D_Cocina_1_Sarten_1',      #19
            'D_Cocina_1_Sarten_2',      #20
            'D_Cocina_1_Sarten_3',      #21
            'D_Cocina_1_Manzana_1',     #22
            'D_Cocina_1_Manzana_2',     #23
            'D_Cocina_1_Manzana_4',     #24
            'D_Cocina_1_Parrilla_1',    #25
            'D_Cocina_1_Pan_1',         #26
            'D_Cocina_1_Tetera_1',      #27
            'D_Cocina_1_Tetera_2',      #28
            'D_Cocina_1_Licuadora_1',   #29
            'D_Cocina_1_Licuadora_2',   #30
            'D_Cocina_1_Llave_1',       #31
            'D_Cocina_1_Escurridor_1',  #32
            'D_Cocina_1_Micro_1'        #33
        ]
        self.scenes = {}
        self.getScences(name0)

    def getScences(self,name1):
        for f in os.listdir(name1):
            if f.split(".")[-1] == "csv":
                csvFile = open(os.path.join(name1,f))
                #print(csvFile.read())
                sceneArray = np.loadtxt(csvFile,delimiter=",").astype(np.int_)
                self.scenes[f] = sceneArray

    def getBOWs(self, data,partitionSize0 = (20,20)):
        bags = {}
        mask = GridMask(sizeOverall0=next(iter(data.values()))[-1].size,sizeXY0=partitionSize0)
        for s in self.scenes.keys():
            bags[s] = []
            shape = self.scenes[s].shape
            for i in range(shape[1]):
                for j in range(shape[0]):
                    if self.scenes[s][j,i] == 0:
                        bags[s].append(BagOfWords(featureArray0=[FeatureMatrix(),],posID0=(i,j)))
                    else:
                        bags[s].append(BagOfWords(featureArray0=data[self.nameMap[self.scenes[s][j,i]-1]],posID0=(i,j)))
        return bags




In [ ]:
sceneMap = SceneMap()
bags_scenes = sceneMap.getBOWs(VCC_Data)
for s in bags_scenes.keys():
    print(f'{s}: ')
    for bow in bags_scenes[s]:
        bow.printCodebook()

---------------TEST SCENES-------------

In [ ]:
sceneMap_Test = SceneMap(name0="Test_Data/Scenes_Test")
bags_scenes_Test = sceneMap_Test.getBOWs(VCC_Data)
for s in bags_scenes_Test.keys():
    print(f'{s}: ')
    for bow in bags_scenes_Test[s]:
        bow.printCodebook()


======================================================= PHC ========================================================

In [ ]:
def buildSceneAdj(bags_scenes0):
    adjH = {}
    adjV = {}

    partitionSize = (20,20)
    for s in bags_scenes0.keys():
        adjH[s] = AdjMatrixH(bows1=bags_scenes0[s],size1=partitionSize,numSymbols1=4)
        adjV[s] = AdjMatrixV(bows1=bags_scenes0[s],size1=partitionSize,numSymbols1=4)
        #print(s)
        adjV[s].printAdj()
        adjH[s].printAdj()
        #print(np.count_nonzero(adjH[s].figureID))
    return adjH,adjV

In [ ]:
def buildSceneConcurrency(bags_scenes0):
    cc = {}

    for s in bags_scenes0.keys():
        cc[s] = ConcurrenceMap(bags_scenes0[s])
        cc[s].printMap()
    return cc

In [ ]:
PHC_adjH,PHC_adjV = buildSceneAdj(bags_scenes)

In [ ]:
PHC_cc = buildSceneConcurrency(bags_scenes)

---------------TEST SCENES ADJ-------------

In [ ]:
PHC_adjH_Test,PHC_adjV_Test = buildSceneAdj(bags_scenes_Test)


In [ ]:
PHC_cc_Test = buildSceneConcurrency(bags_scenes_Test)

======================================================= RSC ========================================================

In [ ]:
class ContextFrame:
    def __init__(self):
        self.map = {}

    def add(self,adjSceneV0,adjSceneH0,concurrenceMap0):
        for bow_id in concurrenceMap0.map.keys():
            if bow_id in self.map.keys():
                c_spaceMap = self.map[bow_id]
                c_spaceMap['U'] = c_spaceMap['U'].union(self.getUp(adjSceneV0,concurrenceMap0.map[bow_id]))
                c_spaceMap['D'] = c_spaceMap['D'].union(self.getDown(adjSceneV0,concurrenceMap0.map[bow_id]))
                c_spaceMap['R'] = c_spaceMap['R'].union(self.getRight(adjSceneH0,concurrenceMap0.map[bow_id]))
                c_spaceMap['L'] = c_spaceMap['L'].union(self.getLeft(adjSceneH0,concurrenceMap0.map[bow_id]))
            else:
                spaceMap = {}
                spaceMap['U'] = self.getUp(adjSceneV0,concurrenceMap0.map[bow_id])
                spaceMap['D'] = self.getDown(adjSceneV0,concurrenceMap0.map[bow_id])
                spaceMap['R'] = self.getRight(adjSceneH0,concurrenceMap0.map[bow_id])
                spaceMap['L'] = self.getLeft(adjSceneH0,concurrenceMap0.map[bow_id])
                self.map[bow_id] = spaceMap
            #self.printCF()

    def getUp(self,adjSceneV1,concurrentBows0):
        up = set()
        for bow in concurrentBows0:
            if bow.posID[1]  < concurrentBows0[0].posID[1]:
                up.add(bow)
        return up

    def getDown(self,adjSceneV1,concurrentBows0):
        down = set()
        for bow in concurrentBows0:
            if bow.posID[1]  > concurrentBows0[0].posID[1]:
                #print(f'pos: {bow.posID[1]}, ref: {concurrentBows0[0].posID[1]}')
                down.add(bow)
        return down

    def getRight(self,adjSceneH1,concurrentBows0):
        right = set()
        for bow in concurrentBows0:
            if bow.posID[0]  > concurrentBows0[0].posID[0]:
                right.add(bow)
        return right

    def getLeft(self,adjSceneH1,concurrentBows0):
        left = set()
        for bow in concurrentBows0:
            if bow.posID[0]  < concurrentBows0[0].posID[0]:
                left.add(bow)
        return left

    def printCF(self):
        s = ""
        print(f'LEN: {len(self.map.keys())}')
        for bow_id in self.map.keys():
            s += f'{bow_id}:\n'
            #print(self.map[bow_id])
            for dir in self.map[bow_id].keys():
                s += f'\t{dir} --> ['
                for c_bow in self.map[bow_id][dir]:
                    s += f'{c_bow.hashID()}||{c_bow.posID}, '
                s += ']\n'
        print(s)


In [ ]:
CF = ContextFrame()
for s in bags_scenes.keys():
    CF.add(PHC_adjV[s],PHC_adjH[s],PHC_cc[s])
    print(s)
CF.printCF()

---------------TEST SCENES CF-------------

In [ ]:
CF_Test = ContextFrame()
for s in bags_scenes_Test.keys():
    CF_Test.add(PHC_adjV_Test[s],PHC_adjH_Test[s],PHC_cc_Test[s])
    print(s)
CF_Test.printCF()


====================================================== MPFC =======================================================

---------------TEST SCENES MATCH-------------

In [ ]:
def getUpBows(cf0,bow_id):
    return cf0.map[bow_id]["U"]

def getDownBows(cf0,bow_id):
    return cf0.map[bow_id]["D"]

def getRightBows(cf0,bow_id):
    return cf0.map[bow_id]["R"]

def getLeftBows(cf0,bow_id):
    return cf0.map[bow_id]["L"]

In [ ]:
def getTarget(pos):
    bow_id_test = ()
    for s in bags_scenes_Test.keys():
        for bow in bags_scenes_Test[s]:
            if bow.posID == pos:
                bow_id_test = bow.hashID()
                break
    return bow_id_test

In [ ]:
def getBows(targetPos, targetID, CF_Test ,ringr=1):
    bow_up = None
    bow_down = None
    bow_right = None
    bow_left = None
    #print(targetID)
    ring_up = ringr
    while ring_up>0:
        for bow in getUpBows(CF_Test,targetID):
            if bow.posID[1] == targetPos[1]-ringr:#<4 and bow_up.posID[1] >=4-ringr:
                bow_up = bow
                #print(bow_up.posID)
                break
        if bow_up != None:
            break
        ring_up -= 1

    ring_down = ringr
    while ring_down>0:
        for bow in getDownBows(CF_Test,targetID):
            if bow.posID[1] == targetPos[1]+ringr:#>4 and bow_down.posID[1] <=4+ringr:
                bow_down = bow
                #print(bow_down.posID)
                break
        if bow_down != None:
            break
        ring_down -= 1

    ring_right = ringr
    while ring_right>0:
        for bow in getRightBows(CF_Test,targetID):
            #if bow != None:
                #print("BOW_RIGHT_ID: " + str(bow.hashID()))
                #print("BOW_RIGHT_POS: " + str(bow.posID))
            if bow.posID[0] ==targetPos[0]+ringr:#>4 and bow_right.posID[0] <=4+ringr:
                bow_right = bow
                #print(bow_right.posID)
                break
        if bow_right != None:
            break
        ring_right -= 1

    ring_left = ringr
    while ring_left>0:
        for bow in getLeftBows(CF_Test,targetID):
            if bow.posID[0] ==targetPos[0]-ringr:#<4 and bow_left.posID[0] >=4-ringr:
                bow_left = bow
                #print(bow_left.posID)
                break
        if bow_left != None:
            break
        ring_left -= 1

    return (bow_up,bow_down,bow_right,bow_left)

In [ ]:
def searchCandidates(targetPos,CF,CF_Test):

    candidate_set = set()
    ringr=1
    risk = 0
    while len(candidate_set)<=0 and ringr<10:
        targetID = getTarget(targetPos)

        bows = getBows(targetPos,targetID,CF_Test,ringr=ringr)
        #print(bows)
        set_up = set()
        set_down = set()
        set_right = set()
        set_left = set()
        candidate_set = set()

        #for bow in bow_up:
        #    set_up = set_up.union(CF.map[bow.hashID()]["D"])
        #for bow in bow_down:
        #    set_down = set_down.union(CF.map[bow.hashID()]["U"])
        #for bow in bow_right:
        #    set_right = set_right.union(CF.map[bow.hashID()]["L"])
        #for bow in bow_left:
        #    set_left = set_left.union(CF.map[bow.hashID()]["R"])
        if bows[0] != None:
            set_up = CF.map[bows[0].hashID()]["D"].union({bows[0]})
        if bows[1] != None:
            set_down = CF.map[bows[1].hashID()]["U"].union({bows[1]})
        if bows[2] != None:
            set_right = CF.map[bows[2].hashID()]["L"].union({bows[2]})
        if bows[3] != None:
            set_left = CF.map[bows[3].hashID()]["R"].union({bows[3]})

        #for set_i in [set_up,set_down,set_right,set_left]:
        #    for set_j in [set_up,set_down,set_right,set_left]:
        #        candidate_set = candidate_set.union(set_i.intersection(set_j))
        #candidate_set = set_up.intersection(set_down).intersection(set_right)#.intersection(set_left)
        #candidate_set =
        #candidate_set = candidate_set.union(set_right.intersection(set_left))
        candidate_sets = []
        candidate_sets.append(set_up.intersection(set_down))
        candidate_sets.append(set_up.intersection(set_down).intersection(set_right))
        candidate_sets.append(set_up.intersection(set_down).intersection(set_right).intersection(set_left))
        candidate_sets.append(set_up.intersection(set_down).intersection(set_left))
        candidate_sets.append(set_up.intersection(set_right))
        candidate_sets.append(set_up.intersection(set_right).intersection(set_left))
        candidate_sets.append(set_up.intersection(set_left))
        candidate_sets.append(set_down.intersection(set_right))
        candidate_sets.append(set_down.intersection(set_right).intersection(set_left))
        candidate_sets.append(set_down.intersection(set_left))
        candidate_sets.append(set_right.intersection(set_left))

        ringr+=1
        risk+=1
        min = 100
        for _set in candidate_sets:
            if len(_set) < min and len(_set) > 0:
                max = len(_set)
                candidate_set = _set


    return tuple([i.featureArray[0].name for i in candidate_set])

In [ ]:
names_test = {}

for bow in list(bags_scenes_Test.values())[0]:
    if not bow.isEmpty():
        names_test[bow.featureArray[0].name] = searchCandidates(bow.posID,CF,CF_Test)

for name in names_test.keys():
    print(f"Object: {name} --> ")
    for candidate in names_test[name]:
        print(f"\t'{candidate}',")

print(names_test.keys())



Object: D_Cocina_1_Tetera_1 --> 
	'F_Cocina_1_Gabinete_2',
	'F_Cocina_1_Gabinete_9',
	'D_Cocina_1_Tetera_1',
	'D_Cocina_1_Tetera_2',
Object: F_Cocina_1_Gabinete_1 --> 
	'F_Cocina_1_Gabinete_1',
	'F_Cocina_1_Gabinete_1',
	'D_Cocina_1_Tetera_2',
	'F_Cocina_1_Gabinete_1',
Object: F_Cocina_1_Gabinete_9 --> 
	'D_Cocina_1_Manzana_1',
	'F_Cocina_1_Estufa_1',
	'D_Cocina_1_Parrilla_1',
	'D_Cocina_1_Sarten_2',
	'F_Cocina_1_Extractor_1',
	'F_Cocina_1_Gabinete_9',
	'F_Cocina_1_Gabinete_2',
Object: D_Cocina_1_Sarten_1 --> 
	'D_Cocina_1_Manzana_1',
	'F_Cocina_1_Estufa_1',
	'D_Cocina_1_Parrilla_1',
	'D_Cocina_1_Sarten_2',
	'F_Cocina_1_Extractor_1',
	'F_Cocina_1_Gabinete_9',
	'F_Cocina_1_Gabinete_2',
Object: F_Cocina_1_Gabinete_2 --> 
	'D_Cocina_1_Manzana_1',
	'F_Cocina_1_Estufa_1',
	'D_Cocina_1_Parrilla_1',
	'D_Cocina_1_Sarten_2',
	'F_Cocina_1_Extractor_1',
	'F_Cocina_1_Gabinete_9',
	'F_Cocina_1_Gabinete_2',
Object: F_Cocina_1_Extractor_1 --> 
	'F_Cocina_1_Estufa_1',
	'D_Cocina_1_Sarten_2',
	'F_Cocin